In [18]:
import numpy as np
import matplotlib.pyplot as plt

In [53]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)
# MSE -> probability

In [48]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [20]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [34]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

### 1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log

In [72]:
def calc_logloss(y, y_pred):
    y_pred = np.array([i for i in y_pred if i !=0])
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [73]:
### Logistic Regression

In [74]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    if i % (iterations / 10) == 0:
        print(i, W, err)
    return W

In [75]:
W = eval_model(X_st, y, iterations=1000, alpha=1e-5)

1000 [ 0.49282757 -0.15007512  0.64748969  1.51727928] 1.2013133809011647


In [71]:
W = eval_model(X_st, y, iterations=1000, alpha=1e-5)

1000 [ 0.49282757 -0.15007512  0.64748969  1.51727928] 1.2013133809011647


In [ ]:
from sklearn.linear_model import LogisticRegression as LR

In [ ]:
np.square(W) / C
lambda = 1 / C

In [ ]:
WOE-преобразование

In [ ]:
# построение ROC-кривой

y_true = [0, 0, 0, 1, 1, 1, 0, 0, 0, 0]
y_pred = [0.1, 0.7, 0.85, 0, 0.9, 0.67, 0.15, 0.25, 0.6, 0.4]
threshold = np.arange(0, 1, 0.01)

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "y_true": y_true,
    "y_pred": y_pred
})

df = df.sort_values(by="y_pred", ascending=False)
df["y_pred"] = df["y_pred"] / 10
df

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [ ]:
fpr, tpr, _ = roc_curve(y_true, y_pred)
plt.plot([0, 1], [0, 1], label="Random", linestyle="--", alpha=0.25)
plt.plot(fpr, tpr, color="green")
plt.scatter(fpr, tpr, color="green")
plt.xlim(0,1)
plt.ylim(0,1)

In [ ]:
roc_auc_score(y_true, y_pred)

In [ ]:
import numpy as np
np.random.randint(X.shape[1])

In [ ]:
X.shape

In [ ]:
precision, recall = 0.7, 0.3

f1_clssic = 2 * precision * recall / (precision + recall)
print(f1_clssic)

In [ ]:
betta = 0.5
f1_weighted = (1 + betta**2) * precision * recall / (betta**2 * precision + recall)
f1_weighted